## Import Library

In [2]:
%pip install tensorflow setuptools pillow scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image
import os

## Setup Dataset

In [5]:
base_dir = 'Dataset/fruit/'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

print(train_dir, val_dir, test_dir)

Dataset/fruit/train Dataset/fruit/val Dataset/fruit/test


## Data Preprocessing

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 560 images belonging to 7 classes.


Found 70 images belonging to 7 classes.
Found 70 images belonging to 7 classes.


## Load Resnet50 Model

In [7]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

## Custom Layer

In [8]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

num_classes = train_generator.num_classes
output_layer = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

## Freeze Base Model Layer

In [9]:
for layer in base_model.layers:
    layer.trainable = False

## Compile Model

In [10]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

## Train Model

In [11]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10


d:\Codes\remote\projects\greeneye-ml\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 0.1359 - loss: 2.3815 - val_accuracy: 0.1429 - val_loss: 1.9397
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.1969 - loss: 1.9111 - val_accuracy: 0.2571 - val_loss: 1.8097
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 0.2027 - loss: 1.8964 - val_accuracy: 0.2000 - val_loss: 1.8028
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.2145 - loss: 1.7689 - val_accuracy: 0.2143 - val_loss: 1.7667
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 0.2960 - loss: 1.7866 - val_accuracy: 0.3429 - val_loss: 1.7504
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.2907 - loss: 1.7767 - val_accuracy: 0.3000 - val_loss: 1.7195
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - accuracy: 0.3518 - loss: 1.6608 - val_accuracy: 0.2286 - val_loss: 1.8474
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - accuracy: 0.3137 - loss: 1.6825 - val_accuracy: 0.3000 - val_loss: 1.7225
Epo

## Evaluate Model

In [12]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy*100:.2f}%')

3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5000 - loss: 1.6710
Test accuracy: 50.00%
